## 학습정리

### 19장. 딥러닝
*
####  19.1 텐서
####  19.2 층 추상화
####  19.3 선형 층
####  19.4 순차적인 층으로 구성된 신경망
####  19.5 손실 함수와 최적화
####  19.6 예시 : XOR문제 다시 풀어 보기
####  19.7 다른 활성화 함수
####  19.8 예시 : Fizz Buzz 다시 풀어보기
####  19.9 Softmax와 Cross-Entropy
####  19.10 드롭아웃
####  19.11 예시 : MNIST
####  19.12 모델 저장 및 불러오기
#### 19.13 더 공부해 보고 싶다면